In [ ]:
"""
Created on Mon Jul 17 14:42 2023

This script is to check the T and S profiles from CMIP

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import cmocean
import glob

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_simple_all = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/SIMPLE/nemo_5km_06161821/'


In [ ]:
mod = 'CNRM-CM6-1' # 'EPM026','EPM031', 'EPM034'
scenario = 'historical'

if scenario == 'historical':
    yystart = 1850
    yyend = 2014
else:
    yystart = 2015
    yyend = 2100

In [ ]:
inputpath_isf='/bettik/burgardc/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_isf+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)


In [ ]:
inputpath_profiles = '/bettik/burgardc/DATA/SUMMER_PAPER/interim/T_S_PROF/CMIP/'+mod+'/'
file_T_list = []
for yy in tqdm(range(yystart,yyend+1)):
    file_T_yy = xr.open_dataset(inputpath_profiles+'T_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(yy)+'.nc')
    file_T_list.append(file_T_yy.assign_coords({'time': yy}))
file_T = xr.concat(file_T_list, dim='time')

file_S_list = []
for yy in tqdm(range(yystart,yyend+1)):
    file_S_yy = xr.open_dataset(inputpath_profiles+'S_mean_prof_50km_contshelf_'+mod+'_'+scenario+'_'+str(yy)+'.nc')
    file_S_list.append(file_S_yy.assign_coords({'time': yy}))
file_S = xr.concat(file_S_list, dim='time')

file_TS = xr.merge([file_T,file_S])

In [ ]:
file_TS_cut_depth = file_TS.where(file_TS.z >= -1*file_isf['front_bot_depth_max'])

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(file_isf.Nisf):

    ax[i] = f.add_subplot(8,8,i+1)

    for tt in file_TS.time:

        T_data = file_TS_cut_depth['thetao'].sel(time = tt, Nisf = kisf)
        ax[i].plot(T_data,T_data.z, color='grey', alpha=alpha_val)
        


        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(-2,2)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'T_profiles_CMIP_'+mod+'_'+scenario+'_all.png', dpi=300)

In [ ]:
regions = ['Weddell','Bellingshausen','Amundsen','Ross','Eastern Ross','East 1','East 2','Dronning Maud Land']
colors = ['red','orange','gold','mediumturquoise','maroon','magenta','cornflowerblue','grey','yellowgreen']

reg_list = file_isf['region'].astype(str)

alpha_val = 0.07

f = plt.figure()
f.set_size_inches(8.25*1.5, 8.25*1.5)

ax={}

leg_hdl = []

i = 0

for kisf in tqdm(file_isf.Nisf):

    ax[i] = f.add_subplot(8,8,i+1)

    for tt in file_TS.time:

        T_data = file_TS_cut_depth['so'].sel(time = tt, Nisf = kisf)
        ax[i].plot(T_data,T_data.z, color='grey', alpha=alpha_val)
        


        #leg, = plt.plot(mean_run,-1*file_TS_mean_all.depth, color=colors[n], linewidth=3)
        #plt.fill_betweenx(-1*mean_run.depth, mean_run - std_run, mean_run + std_run, -1*mean_run.depth, alpha=0.2, color=colors[n])
        #leg_hdl.append(leg)

    ax[i].set_title(str(file_isf_orig['isf_name'].sel(Nisf=kisf).values))
    #ax[i].set_xlim(33,35)

    i = i+1


f.subplots_adjust(bottom=0.05, wspace=0.1)

f.tight_layout()
sns.despine()
#f.savefig(plot_path + 'S_profiles_CMIP_'+mod+'_'+scenario+'_all.png', dpi=300)